In [1]:
import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras.layers import Dense, Flatten, Conv2D, Dropout, MaxPooling2D

In [2]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

x_train = x_train[..., tf.newaxis].astype("float32")
x_test = x_test[..., tf.newaxis].astype("float32")

In [3]:
train_ds = tf.data.Dataset.from_tensor_slices(
    (x_train, y_train)).shuffle(10000).batch(32)

test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)


In [11]:
class CustomModel(Model):
  def __init__(self):
    super(CustomModel, self).__init__()
    self.conv1 = Conv2D(filters=32, kernel_size=3, activation=tf.nn.relu)
    self.max_pool = MaxPooling2D(pool_size=(2, 2), strides=2)
    self.flatten = Flatten()
    self.fc1 = Dense(128, activation=tf.nn.relu)
    self.dropout = Dropout(rate=0.2)
    self.fc2 = Dense(10, activation=tf.nn.softmax)

  def call(self, x):
    x = self.conv1(x)
    x = self.max_pool(x)
    x = self.flatten(x)
    x = self.fc1(x)
    x = self.dropout(x)
    
    return self.fc2(x)

model = CustomModel()

In [12]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False)
optimizer = tf.keras.optimizers.Adam()

In [13]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

In [14]:
@tf.function
def train_step(images, labels):
    with tf.GradientTape() as tape:
        predictions = model(images, training=True)
        loss = loss_object(labels, predictions)

    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    train_loss(loss)
    train_accuracy(labels, predictions)

In [15]:
@tf.function
def test_step(images, labels):
    predictions = model(images, training=False)
    t_loss = loss_object(labels, predictions)

    test_loss(t_loss)
    test_accuracy(labels, predictions)

In [16]:
@tf.function
def reset_gradient():
    train_loss.reset_states()
    train_accuracy.reset_states()
    
    test_loss.reset_states()
    test_accuracy.reset_states()

In [17]:
EPOCHS = 5

for epoch in range(EPOCHS):
    reset_gradient()

    for images, labels in train_ds:
        train_step(images, labels)

    for test_images, test_labels in test_ds:
        test_step(test_images, test_labels)

    print(
        f'Epoch {epoch + 1}, '
        f'Loss: {train_loss.result()}, '
        f'Accuracy: {train_accuracy.result() * 100}, '
        f'Test Loss: {test_loss.result()}, '
        f'Test Accuracy: {test_accuracy.result() * 100}'
    )


Epoch 1, Loss: 0.18205764889717102, Accuracy: 94.6449966430664, Test Loss: 0.062325432896614075, Test Accuracy: 98.0199966430664
Epoch 2, Loss: 0.06831832230091095, Accuracy: 97.93999481201172, Test Loss: 0.045215122401714325, Test Accuracy: 98.55999755859375
Epoch 3, Loss: 0.0460178516805172, Accuracy: 98.54166412353516, Test Loss: 0.042453423142433167, Test Accuracy: 98.50999450683594
Epoch 4, Loss: 0.03616693243384361, Accuracy: 98.8499984741211, Test Loss: 0.04066595435142517, Test Accuracy: 98.66999816894531
Epoch 5, Loss: 0.027779053896665573, Accuracy: 99.086669921875, Test Loss: 0.038908425718545914, Test Accuracy: 98.68000030517578
